In [ ]:
import pandas as pd
from keras.layers import LSTM, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation
from keras.models import Model
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import numpy as np
import os
import fnmatch

## Model

In [ ]:
def get_model(input_shape, output_shape, lr = 0.00001):
    inputs = Input(shape=input_shape)
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Bidirectional(LSTM(1024, return_sequences=True))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    return model

## Data

In [ ]:
def generator(path, mode, num_batches):
    counter = 0
    while True:
        if counter >= num_batches:
            counter = 0

        features = np.load(path + 'features_' + mode + '_' + str(counter) + '.npy')
        classes = np.load(path + 'classes_' + mode + '_' + str(counter) + '.npy')
        
        counter += 1
        yield features, classes
        
def get_metadata(path):
    features = np.load(path + 'features_train_0.npy')
    input_shape = features.shape[1:]
    classes = np.load(path + 'classes_train_0.npy')
    num_classes = classes.shape[2]
    steps_per_epoch = len(fnmatch.filter(os.listdir(path),'*features_train_*'))
    validation_steps = len(fnmatch.filter(os.listdir(path),'*features_dev_*'))
    return input_shape, num_classes, steps_per_epoch, validation_steps

## Training

In [ ]:
input_shape, num_classes, steps_per_epoch, validation_steps = get_metadata('./features/lstm/2_steps/')

In [ ]:
print('Input shape:', input_shape)
print('Number of classes:', num_classes)
print('Steps per epoch:', steps_per_epoch) 
print('Validation steps:', validation_steps)

In [ ]:
model = get_model(input_shape, num_classes)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(
    generator('./features/lstm/2_steps/', 'train', steps_per_epoch),
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=generator('./features/lstm/2_steps/', 'dev', validation_steps),
    validation_steps=validation_steps,
)